In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(arguments[0], signed=direction(arguments[0])) 
    event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(arguments[1], signed=direction(arguments[1])) 
    event['details']['sqrt_price_x96'] = Conversion().convert_int256_bytes_to_int(arguments[2]) 
    event['details']['liquidity'] = Conversion().convert_int256_bytes_to_int(arguments[3]) 
    event['details']['tick'] = Conversion().convert_int256_bytes_to_int(arguments[4], signed=direction(arguments[4])) 
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,316,270 tx:0xf3ae2e5da6cd94c6e3ecd476898f7789eb4f777360ad7da1d743ed2db03e1ec6
Swap at block:60,316,270 tx:0xabc1f60d1e32b3e9052983b990b660144a1a70d09d5ce2264589b65222bbb294
Swap at block:60,316,271 tx:0x2ffd89ca97c3099a17d40922ed4cd000bad01e50320694be2ab068c662e4b879
Swap at block:60,316,271 tx:0xe2e452b7f8eb714ff2e47ad07c95fba78fd7666d19a88c5dd75a6e1e62a178a8
Swap at block:60,316,271 tx:0x25d724750f5ba803f6cc1ef9b05772b4671984be5810205fb68c3a23457e6c2c
Swap at block:60,316,271 tx:0xf488c3ad19a014d80516f57f1e3dc162b4b7eaf930ba24640247f4b56e18a02b
Swap at block:60,316,271 tx:0xd5b47e19e6789055695077ff22eb8cb5b33b3daf67a890d5942a3b77c64c85f7
Swap at block:60,316,271 tx:0x08312a4985b2d5d1337936e5b9e7dc405dbc14b400f8136b333fd7e1db09f1b5
Swap at block:60,316,271 tx:0x48f8b1135a3538dfc352931549fc6c6f89ef459f7e8ce6d8a0e7c2b9c6daa1a6
Swap at block:60,316,271 tx:0x6de9aebf52f917e05b3820fafe4dcb9697dcd0952d18e5bff5e0833fa36a8853
Swap at block:60,316,271 tx:0x2d6d99759dcb91cf0c90

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,swap,uniswap_v3,0xc56ddb5c93b8e92b9409dce43a9169aa643495b8,0xf3ae2e5da6cd94c6e3ecd476898f7789eb4f777360ad...,60316270,1723069992,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv3pool,swap,uniswap_v3,0xa374094527e1673a86de625aa59517c5de346d32,0xabc1f60d1e32b3e9052983b990b660144a1a70d09d5c...,60316270,1723069992,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv3pool,swap,uniswap_v3,0x45dda9cb7c25131df268515131f647d726f50608,0x2ffd89ca97c3099a17d40922ed4cd000bad01e503206...,60316271,1723069994,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv3pool,swap,uniswap_v3,0x9ceff2f5138fc59eb925d270b8a7a9c02a1810f2,0xe2e452b7f8eb714ff2e47ad07c95fba78fd7666d19a8...,60316271,1723069994,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv3pool,swap,uniswap_v3,0xa374094527e1673a86de625aa59517c5de346d32,0x25d724750f5ba803f6cc1ef9b05772b4671984be5810...,60316271,1723069994,{'web3_type': <class 'web3._utils.datatypes.Sw...
...,...,...,...,...,...,...,...,...,...
149,polygon,uniswapv3pool,swap,uniswap_v3,0xec15624fbb314eb05baad4ca49b7904c0cb6b645,0x5d239c10ea68ee1a6bf58e93edf357a3cd82b0ac7221...,60316279,1723070012,{'web3_type': <class 'web3._utils.datatypes.Sw...
150,polygon,uniswapv3pool,swap,uniswap_v3,0x781067ef296e5c4a4203f81c593274824b7c185d,0x28579be0e8464b04a11a8c7a9afc5cdef0b96eaa4320...,60316279,1723070012,{'web3_type': <class 'web3._utils.datatypes.Sw...
151,polygon,uniswapv3pool,swap,uniswap_v3,0x6b75f2189f0e11c52e814e09e280eb1a9a8a094a,0x0fc23a07202a824c93026598e4e9957dd8c93cf2cde8...,60316279,1723070012,{'web3_type': <class 'web3._utils.datatypes.Sw...
152,polygon,uniswapv3pool,swap,uniswap_v3,0x9b08288c3be4f62bbf8d1c20ac9c5e6f9467d8b7,0x06617b7b942dd0b5cd9525f3161f8a5f2f14a9318866...,60316279,1723070012,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0xc56ddb5c93b8e92b9409dce43a9169aa643495b8',
  'tx_hash': '0xf3ae2e5da6cd94c6e3ecd476898f7789eb4f777360ad7da1d743ed2db03e1ec6',
  'blk_num': 60316270,
  'timestamp': 1723069992,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xE592427A0AEce92De3Edee1F18E0157C05861564',
   'token1': '0xa62195eAbDBf594664d3591a982064090bB52c87',
   'amount0': 1281787465481818600000,
   'amount1': -13228037,
   'sqrt_price_x96': 8060141775953234899179,
   'liquidity': 973003072615393129,
   'tick': -322035}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0xa374094527e1673a86de625aa59517c5de346d32',
  'tx_hash': '0xabc1f60d1e32b3e9052983b990b660144a1a70d09d5ce2264589b65222bbb294',
  'blk_num': 60316270,
  'timestamp': 1723069992,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x80